# Trino data management demo sample
Copyright (C) 2021 OS-Climate

This sample shows:
* How Arrow and Parquet can be used to provide direct support for persisting dataFrames and metadata together in highly portable and performant files. This mechanism allows us to manage data ingestion and metadata in a transactional and integrated way.
* Simple metadata management for data set-level and field-level information inside dedicated Trino schema and tables


Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

In [1]:
# pip install boto3
# pip install python-dotenv
# pip install trino
# pip install pandas
# pip install pyarrow
# pip install osc-ingest-tools

In [2]:
from dotenv import dotenv_values, load_dotenv
from osc_ingest_trino import *
import os
import pathlib
import boto3
import pandas as pd
import uuid
import pyarrow as pa
import pyarrow.parquet as pq
import json
import trino
import io

Load Environment Variables

In [3]:
dotenv_dir = os.environ.get('CREDENTIAL_DOTENV_DIR', os.environ.get('PWD', '/opt/app-root/src'))
dotenv_path = pathlib.Path(dotenv_dir) / 'credentials.env'
if os.path.exists(dotenv_path):
    load_dotenv(dotenv_path=dotenv_path,override=True)

Create a simple data frame for testing

In [4]:
# initialise data of lists of powerplants and capacity
data = {'name':['Albas', 'Albi', 'Albias', 'Allaire'],
        'capacity_gwh':[1.8, 6.70448, 2.41, 8.2]}
# Create DataFrame
df = pd.DataFrame(data)
# Add a unique identifier to the data set
uid = str(uuid.uuid4())
df['uuid'] = uid
# Print the output
df = df.convert_dtypes()

In [5]:
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   name          4 non-null      string 
 1   capacity_gwh  4 non-null      Float64
 2   uuid          4 non-null      string 
dtypes: Float64(1), string(2)
memory usage: 228.0 bytes


Convert the DataFrame to an Arrow table using PyArrow and inspect the table’s metadata property 

In [6]:
table = pa.Table.from_pandas(df)
print(table.schema.metadata)

{b'pandas': b'{"index_columns": [{"kind": "range", "name": null, "start": 0, "stop": 4, "step": 1}], "column_indexes": [{"name": null, "field_name": null, "pandas_type": "unicode", "numpy_type": "object", "metadata": {"encoding": "UTF-8"}}], "columns": [{"name": "name", "field_name": "name", "pandas_type": "unicode", "numpy_type": "string", "metadata": null}, {"name": "capacity_gwh", "field_name": "capacity_gwh", "pandas_type": "float64", "numpy_type": "Float64", "metadata": null}, {"name": "uuid", "field_name": "uuid", "pandas_type": "unicode", "numpy_type": "string", "metadata": null}], "creator": {"library": "pyarrow", "version": "5.0.0"}, "pandas_version": "1.3.3"}'}


Create custom meta data and key

In [7]:
custom_meta_content = {
    'dataset_key': 'gppd',
    'title': 'Global Power Plant Database',
    'description': 'A comprehensive, global, open source database of power plants',
    'version': '1.3.0',
    'release_date': '20210602',
    'fields': [
    {
        'field_name': 'name',
        'definition': 'name of the power plant',
        'type': 'text'
    },
    {
        'field_name': 'capacity_gwh',
        'definition': 'electricity generation in gigawatt-hours',
        'type': 'number'
    }]
}
custom_meta_key = 'metaset'

Construct the combined metadata by merging existing table metadata and custom metadata.
Note: The metadata content must be JSON serialisable and encoded as bytes; the metadata key must also be encoded as bytes.

In [8]:
custom_meta_json = json.dumps(custom_meta_content)
existing_meta = table.schema.metadata
combined_meta = {
    custom_meta_key.encode(): custom_meta_json.encode(),
    **existing_meta
}

Replace the original table metadata by the combined metadata in the Arrow table. This table now contains both the custom metadata and the Pandas metadata. 

In [9]:
table = table.replace_schema_metadata(combined_meta)
print(table.schema.metadata)

{b'metaset': b'{"dataset_key": "gppd", "title": "Global Power Plant Database", "description": "A comprehensive, global, open source database of power plants", "version": "1.3.0", "release_date": "20210602", "fields": [{"field_name": "name", "definition": "name of the power plant", "type": "text"}, {"field_name": "capacity_gwh", "definition": "electricity generation in gigawatt-hours", "type": "number"}]}', b'pandas': b'{"index_columns": [{"kind": "range", "name": null, "start": 0, "stop": 4, "step": 1}], "column_indexes": [{"name": null, "field_name": null, "pandas_type": "unicode", "numpy_type": "object", "metadata": {"encoding": "UTF-8"}}], "columns": [{"name": "name", "field_name": "name", "pandas_type": "unicode", "numpy_type": "string", "metadata": null}, {"name": "capacity_gwh", "field_name": "capacity_gwh", "pandas_type": "float64", "numpy_type": "Float64", "metadata": null}, {"name": "uuid", "field_name": "uuid", "pandas_type": "unicode", "numpy_type": "string", "metadata": nul

Open an S3 client connection and save the dataframe as a parquet file so that dataFrame & metadata are now coupled together

In [10]:
# Create an S3 client
s3 = boto3.client(
    service_name="s3",
    endpoint_url=os.environ['S3_DEV_ENDPOINT'],
    aws_access_key_id=os.environ['S3_DEV_ACCESS_KEY'],
    aws_secret_access_key=os.environ['S3_DEV_SECRET_KEY'],
)

In [11]:
schemaname = 'wri'
tablename = 'gppd'
pq.write_table(
    table,
    '/tmp/{tname}.parquet'.format(tname=tablename)
)
s3.upload_file(
    Bucket=os.environ['S3_DEV_BUCKET'],
    Key='trino/{sname}/{tname}/{tname}.parquet'.format(sname=schemaname,tname=tablename),
    Filename='/tmp/{tname}.parquet'.format(tname=tablename)
)

Convert custom metadata content in json format into Pandas DataFrame

In [12]:
df_meta_fields = pd.json_normalize(custom_meta_content, record_path =['fields'], meta=['dataset_key']).convert_dtypes()
df_meta_fields

,field_name,definition,type,dataset_key
0,name,name of the power plant,text,gppd
1,capacity_gwh,electricity generation in gigawatt-hours,number,gppd


In [13]:
df_meta_fields.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   field_name   2 non-null      string
 1   definition   2 non-null      string
 2   type         2 non-null      string
 3   dataset_key  2 non-null      string
dtypes: string(4)
memory usage: 192.0 bytes


In [14]:
df_meta_table = pd.json_normalize(custom_meta_content, max_level=0)
df_meta_table.drop('fields', inplace=True, axis=1)
df_meta_table['schema'] = schemaname
df_meta_table = df_meta_table.convert_dtypes()
df_meta_table

,dataset_key,title,description,version,release_date,schema
0,gppd,Global Power Plant Database,"A comprehensive, global, open source database ...",1.3.0,20210602,wri


In [15]:
df_meta_table.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   dataset_key   1 non-null      string
 1   title         1 non-null      string
 2   description   1 non-null      string
 3   version       1 non-null      string
 4   release_date  1 non-null      string
 5   schema        1 non-null      string
dtypes: string(6)
memory usage: 176.0 bytes


Upload parquet file for metadata storage in Trino

In [16]:
tablename_meta_table = 'tables'
tablename_meta_fields ='fields'
df_meta_table.to_parquet('/tmp/{tname}.parquet'.format(tname=tablename_meta_table), index=False)
s3.upload_file(
    Bucket=os.environ['S3_DEV_BUCKET'],
    Key='trino/{sname}/{tname}/{tname}.parquet'.format(sname='metastore',tname=tablename_meta_table),
    Filename='/tmp/{tname}.parquet'.format(tname=tablename_meta_table)
)
df_meta_fields.to_parquet('/tmp/{tname}.parquet'.format(tname=tablename_meta_fields), index=False)
s3.upload_file(
    Bucket=os.environ['S3_DEV_BUCKET'],
    Key='trino/{sname}/{tname}/{tname}.parquet'.format(sname='metastore',tname=tablename_meta_fields),
    Filename='/tmp/{tname}.parquet'.format(tname=tablename_meta_fields)
)

Restore data and metadata

In [17]:
# Read the Parquet file into an Arrow table
obj = s3.get_object(
    Bucket=os.environ['S3_DEV_BUCKET'], 
    Key='trino/wri/{tname}/{tname}.parquet'.format(tname=tablename)
)
restored_table = pq.read_table(io.BytesIO(obj['Body'].read()))
# Call the table’s to_pandas conversion method to restore the dataframe
# This operation uses the Pandas metadata to reconstruct the dataFrame under the hood
restored_df = restored_table.to_pandas()
# The custom metadata is accessible via the Arrow table’s metadata object
# Use the custom metadata key used earlier (taking care to once again encode the key as bytes)
restored_meta_json = restored_table.schema.metadata[custom_meta_key.encode()]
# Deserialize the json string to get back metadata
restored_meta = json.loads(restored_meta_json)

In [18]:
restored_df

,name,capacity_gwh,uuid
0,Albas,1.8,0a8d1a11-dabe-4b86-b562-2b586baf7531
1,Albi,6.70448,0a8d1a11-dabe-4b86-b562-2b586baf7531
2,Albias,2.41,0a8d1a11-dabe-4b86-b562-2b586baf7531
3,Allaire,8.2,0a8d1a11-dabe-4b86-b562-2b586baf7531


In [19]:
restored_meta

{'dataset_key': 'gppd',
 'title': 'Global Power Plant Database',
 'description': 'A comprehensive, global, open source database of power plants',
 'version': '1.3.0',
 'release_date': '20210602',
 'fields': [{'field_name': 'name',
   'definition': 'name of the power plant',
   'type': 'text'},
  {'field_name': 'capacity_gwh',
   'definition': 'electricity generation in gigawatt-hours',
   'type': 'number'}]}

Open a Trino connection using JWT for authentication

In [20]:
conn = trino.dbapi.connect(
    host=os.environ['TRINO_HOST'],
    port=int(os.environ['TRINO_PORT']),
    user=os.environ['TRINO_USER'],
    http_scheme='https',
    auth=trino.auth.JWTAuthentication(os.environ['TRINO_PASSWD']),
    verify=True,
)
cur = conn.cursor()

In [21]:
# Show available schemas to ensure trino connection is set correctly
cur.execute('show schemas in osc_datacommons_dev')
cur.fetchall()

[['aicoe_osc_demo'],
 ['company_data'],
 ['default'],
 ['defaultschema1'],
 ['demo'],
 ['eje'],
 ['epacems'],
 ['epacems_y95_al'],
 ['information_schema'],
 ['metastore'],
 ['osc_corp_data'],
 ['physical_risk_project'],
 ['pudl'],
 ['rmi_utility_transition_hub'],
 ['team1'],
 ['team2'],
 ['testaccessschema1'],
 ['urgentem'],
 ['wri'],
 ['wri_gppd'],
 ['wri_gppd_md']]

Create ingestion schema based on source data name and remove old tables if necessary

In [22]:
cur.execute('create schema if not exists osc_datacommons_dev.wri')
cur.fetchall()

[[True]]

In [23]:
cur.execute('drop table if exists osc_datacommons_dev.wri.' + tablename)
cur.fetchall()

[[True]]

In [24]:
schema = create_table_schema_pairs(df)
tabledef = """create table if not exists osc_datacommons_dev.{sname}.{tname}(
{schema}
) with (
    format = 'parquet',
    external_location = 's3a://{bucket}/trino/{sname}/{tname}/'
)""".format(schema=schema,bucket=os.environ['S3_DEV_BUCKET'],sname=schemaname,tname=tablename)
print(tabledef)

# tables created externally may not show up immediately in cloud-beaver
cur.execute(tabledef)
cur.fetchall()

create table if not exists osc_datacommons_dev.wri.gppd(
    name varchar,
    capacity_gwh double,
    uuid varchar
) with (
    format = 'parquet',
    external_location = 's3a://ocp-odh-os-demo-s3/trino/wri/gppd/'
)


[[True]]

In [25]:
dataset_query = ('SELECT * FROM osc_datacommons_dev.{sname}.{tname} limit 10').format(sname=schemaname,tname=tablename)
print(dataset_query)
cur.execute(dataset_query)
cur.fetchall()

SELECT * FROM osc_datacommons_dev.wri.gppd limit 10


[['Albas', 1.8, '0a8d1a11-dabe-4b86-b562-2b586baf7531'],
 ['Albi', 6.70448, '0a8d1a11-dabe-4b86-b562-2b586baf7531'],
 ['Albias', 2.41, '0a8d1a11-dabe-4b86-b562-2b586baf7531'],
 ['Allaire', 8.2, '0a8d1a11-dabe-4b86-b562-2b586baf7531']]

Create metadata table for schema / dataset level information

In [26]:
schema_meta_table = create_table_schema_pairs(df_meta_table)
tabledef = """create table if not exists osc_datacommons_dev.{sname}.{tname}(
{schema}
) with (
    format = 'parquet',
    external_location = 's3a://{bucket}/trino/{sname}/{tname}/'
)""".format(schema=schema_meta_table,bucket=os.environ['S3_DEV_BUCKET'],sname='metastore',tname=tablename_meta_table)
print(tabledef)

# tables created externally may not show up immediately in cloud-beaver
cur.execute(tabledef)
cur.fetchall()

create table if not exists osc_datacommons_dev.metastore.tables(
    dataset_key varchar,
    title varchar,
    description varchar,
    version varchar,
    release_date varchar,
    schema varchar
) with (
    format = 'parquet',
    external_location = 's3a://ocp-odh-os-demo-s3/trino/metastore/tables/'
)


[[True]]

In [27]:
meta_query_table = ('SELECT * FROM osc_datacommons_dev.{sname}.{tname} limit 10').format(sname='metastore',tname=tablename_meta_table)
print(meta_query_table)
cur.execute(meta_query_table)
cur.fetchall()

SELECT * FROM osc_datacommons_dev.metastore.tables limit 10


[['gppd',
  'Global Power Plant Database',
  'A comprehensive, global, open source database of power plants',
  '1.3.0',
  '20210602',
  'wri']]

Create metadata table for fields information

In [28]:
schema_meta_fields = create_table_schema_pairs(df_meta_fields)
tabledef = """create table if not exists osc_datacommons_dev.{sname}.{tname}(
{schema}
) with (
    format = 'parquet',
    external_location = 's3a://{bucket}/trino/{sname}/{tname}/'
)""".format(schema=schema_meta_fields,bucket=os.environ['S3_DEV_BUCKET'],sname='metastore',tname=tablename_meta_fields)
print(tabledef)

# tables created externally may not show up immediately in cloud-beaver
cur.execute(tabledef)
cur.fetchall()

create table if not exists osc_datacommons_dev.metastore.fields(
    field_name varchar,
    definition varchar,
    type varchar,
    dataset_key varchar
) with (
    format = 'parquet',
    external_location = 's3a://ocp-odh-os-demo-s3/trino/metastore/fields/'
)


[[True]]

In [29]:
meta_query_fields = ('SELECT * FROM osc_datacommons_dev.{sname}.{tname} limit 10').format(sname='metastore',tname=tablename_meta_fields)
print(meta_query_fields)
cur.execute(meta_query_fields)
cur.fetchall()

SELECT * FROM osc_datacommons_dev.metastore.fields limit 10


[['name', 'name of the power plant', 'text', 'gppd'],
 ['capacity_gwh',
  'electricity generation in gigawatt-hours',
  'number',
  'gppd']]